In [36]:
import tensorflow as tf

import numpy as np

from keras.utils import to_categorical
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Input, Lambda, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image

import tensorflow_datasets as tfds


In [37]:
batch_size = 32
epochs = 5
filter_size = 3
group_1_dim = 32
group_2_dim = 64
group_3_dim = 128
dense_dim = 1024

dropout_rate = 0.2

In [42]:
train_ds = tf.keras.utils.image_dataset_from_directory(r"C:\Users\haoxu\Documents\ecec147\cifar10_tensorflow\img_uniform_split\train", 
                                                 labels='inferred', 
                                                 image_size=(64,64), 
                                                 batch_size=batch_size,
                                                 validation_split=0.2,
                                                 subset="training",
                                                 seed=123)

val_ds = tf.keras.utils.image_dataset_from_directory(r"C:\Users\haoxu\Documents\ecec147\cifar10_tensorflow\img_uniform_split\train", 
                                                 labels='inferred', 
                                                 image_size=(64,64), 
                                                 batch_size=batch_size,
                                                 validation_split=0.2,
                                                 subset="validation",
                                                 seed=123)

test_ds = tf.keras.utils.image_dataset_from_directory(r"C:\Users\haoxu\Documents\ecec147\cifar10_tensorflow\img_uniform_split\test", 
                                                 labels='inferred', 
                                                 image_size=(64,64), 
                                                 batch_size=batch_size)

Found 89663 files belonging to 10 classes.
Using 71731 files for training.
Found 89663 files belonging to 10 classes.
Using 17932 files for validation.
Found 9966 files belonging to 10 classes.


In [41]:
# (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [56]:
def preprocess_data(X, Y):
  """
  Method:
      pre-processes the data for your model.
  """
  X_p = tf.keras.applications.inception_v3.preprocess_input(X)
  Y_p = tf.one_hot(Y, 10)

  return X_p, Y_p

# X_train_p, Y_train_p = preprocess_data(x_train, y_train)
# X_test_p, Y_test_p = preprocess_data(x_test, y_test)
train_ds = train_ds.map(preprocess_data)
val_ds = val_ds.map(preprocess_data)
test_ds = test_ds.map(preprocess_data)

AttributeError: in user code:

    File "C:\Users\haoxu\AppData\Local\Temp\ipykernel_14020\3010396901.py", line 7, in preprocess_data  *
        Y_p = Y.flatten()

    AttributeError: 'SymbolicTensor' object has no attribute 'flatten'


In [49]:
base_inception = InceptionV3(include_top=False,
                             input_shape=(299, 299, 3))

input_layer = Input(shape=(64, 64, 3))

resizing_layer = Lambda(lambda image:                     \
                tf.keras.preprocessing.image.smart_resize(image, \
                (299, 299)))(input_layer)

inception_layers = base_inception(resizing_layer, training=False)

glob_pooling = GlobalAveragePooling2D()(inception_layers)
layer_i = Dense(500, activation='relu')(glob_pooling)
dropout_layer = Dropout(0.3)(layer_i)
output_layer =  Dense(10, activation='softmax')(dropout_layer)

model = Model(inputs=input_layer, outputs=output_layer)

In [50]:
# Freeze the base network
base_inception.trainable = False

model.compile(tf.keras.optimizers.Adam(learning_rate=.0001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

history = model.fit(
        train_ds,
        validation_data=val_ds,
        batch_size=300,
        epochs=4,
        verbose=1)

results = model.evaluate(test_ds)

Epoch 1/4


ValueError: in user code:

    File "C:\Users\haoxu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\engine\training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\haoxu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\engine\training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\haoxu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\engine\training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\haoxu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\engine\training.py", line

In [ ]:
for layer in base_inception.layers[:164]:
  layer.trainable=False

for layer in base_inception.layers[164:]:
  layer.trainable=True

my_optimizer = tf.keras.optimizers.Adam(1e-5)
model.compile(optimizer=my_optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(
        X_train_p,
        Y_train_p,
        validation_data=(X_test_p, Y_test_p),
        batch_size=300,
        epochs=4,
        verbose=1)

results = model.evaluate(X_test_p, Y_test_p)

Epoch 1/4
167/167 [==============================] - 1238s 7s/step - loss: 0.3673 - accuracy: 0.8762 - val_loss: 0.3833 - val_accuracy: 0.8678
Epoch 2/4
 54/167 [========>.....................] - ETA: 11:39 - loss: 0.3602 - accuracy: 0.8786

KeyboardInterrupt: 